# Canopy height
Input: 
- Forest4model_v1_Canopy_height

Aim:
- Visualising the distribution of canopy height in sample countries
- Calculating the average canopy height in EU countries
- Differentiating the canopy height to 10 bin classes (fraction data is ranging from 0 to 1)
- Calculating percentage share of each canopy height class over country's total forest cover

Output:
- A dataframe containing average canopy height in 32 EU countries
- A dataframe containing percentage share of canopy height classes over country's total forest cover

In [1]:
# Vector data
import geopandas as gpd

# Tabular data 
import pandas as pd
import numpy as np

# Raster array
import xarray as xr

# Plotting
import matplotlib.pyplot as plt

# System
from pathlib import Path

# Import functions from geo_tools module
from geo_tools import utils, forest_barplot, plot_iso_forest

# Reloads module
%load_ext autoreload
%autoreload 2

# Output directory

In [2]:
# Specify the desired output folder path for processed dataset
output_data_dir = Path.cwd() / ".." / "output_data"

# Check if the output folder exists, and create if not
Path.mkdir(output_data_dir, exist_ok=True, parents=True)

In [3]:
# Specify the desired output folder path for figures
output_figure_dir = Path.cwd() / ".." / "output_figures"

# Check if the output folder exists, and create if not
Path.mkdir(output_figure_dir, exist_ok=True, parents=True)

# A list of ISO3 codes of EU countries

In [4]:
eu_iso3 = pd.read_csv(output_data_dir/"Country_Codes_and_Names_ISO3.csv")

eu_iso = eu_iso3["ISO3"]

# Plotting data distribution

In [5]:
# Testing plot_iso_forest for many countries at once
for iso in ["FRA", "DEU", "SWE"]:
    plot_iso_forest.plot_forest(
        iso_code=iso,
        xarray_id="Canopy_height"
    )

KeyboardInterrupt: 

# Average canopy height

In [6]:
canopy_avg = []

for iso in eu_iso:
    iso_canopy_avg = plot_iso_forest.average_values(
        iso_code=iso,
        xarray_id="Canopy_height",
        forest_layer_year=2020
    )

    canopy_avg.append(iso_canopy_avg)

canopy_avg_df = pd.concat(canopy_avg)

In [10]:
canopy_avg_df

,Region,Variable,Var_uid,Year,Unit,Value
0,BGR,Canopy height,canopy_height,2020,Meters,17.097071
0,CYP,Canopy height,canopy_height,2020,Meters,9.484508
0,DNK,Canopy height,canopy_height,2020,Meters,18.072762
0,IRL,Canopy height,canopy_height,2020,Meters,16.207054
0,EST,Canopy height,canopy_height,2020,Meters,20.063599
0,AUT,Canopy height,canopy_height,2020,Meters,20.460241
0,CZE,Canopy height,canopy_height,2020,Meters,19.828314
0,FIN,Canopy height,canopy_height,2020,Meters,19.259748
0,FRA,Canopy height,canopy_height,2020,Meters,16.881876
0,DEU,Canopy height,canopy_height,2020,Meters,20.269926


# Differentiating classes for canopy height

In [16]:
%%time

# Import forest cover data
forest_cover = pd.read_csv(output_data_dir/"iso_forest_cover_stats.csv").drop(["Unnamed: 0"], axis=1)

# Template to store country results in Mha
canopy_mha = []

# Template to store country results in percentage share
canopy_perc = []

for iso in ["DEU"]:

    # Calculate areas for each class 
    iso_canopy_class = utils.define_class(
        iso_code=iso,
        xarray_id="Canopy_height",
        forest_layer_year=2020,
        start_step=0,
        end_step=5.0,
        delta_diff=5.0,
        max_val=35.0
    )
    canopy_mha.append(iso_canopy_class)

    # Get forest cover area within the country
    iso_forest_cover = forest_cover.loc[forest_cover.Region == iso]
    value_to_copy = iso_forest_cover.iloc[0, 5]

    # Copy forest cover to iso_fraction_class
    iso_canopy_class[f"{iso}_forest_cover_Mha"] = value_to_copy

    # Calculate percentage share of fraction classes over total forest cover
    iso_canopy_class["Percentage share"] = iso_canopy_class["Value"] / iso_canopy_class[f"{iso}_forest_cover_Mha"]

    # Creating final dataset
    iso_canopy_class_perc = iso_canopy_class.drop(["Unit", "Value", f"{iso}_forest_cover_Mha"], axis=1)

    iso_canopy_class_perc.insert(
        loc=3,
        column="Unit",
        value="Percentage share"
    )

    iso_canopy_class_perc.rename(
        columns={"Percentage share" : "Value"},
        inplace=True
    )

    canopy_perc.append(iso_canopy_class_perc)

canopy_perc_df = pd.concat(canopy_perc)

CPU times: user 2.46 s, sys: 666 ms, total: 3.13 s
Wall time: 3.13 s


In [17]:
canopy_perc_df[canopy_perc_df.Region=="DEU"]

,Region,Variable,Var_uid,Unit,Year,Value
0,DEU,Canopy height >0 to <= 5.0,canopy_height,Percentage share,2020,0.004247
1,DEU,Canopy height >5.0 to <= 10.0,canopy_height,Percentage share,2020,0.067348
2,DEU,Canopy height >10.0 to <= 15.0,canopy_height,Percentage share,2020,0.095298
3,DEU,Canopy height >15.0 to <= 20.0,canopy_height,Percentage share,2020,0.225936
4,DEU,Canopy height >20.0 to <= 25.0,canopy_height,Percentage share,2020,0.488505
5,DEU,Canopy height >25.0 to <= 30.0,canopy_height,Percentage share,2020,0.116272
6,DEU,Canopy height >30.0 to <= 35.0,canopy_height,Percentage share,2020,0.000094


In [44]:
# Exporting the dataset
canopy_height_df = pd.concat([canopy_avg_df, canopy_perc_df])

canopy_height_df.to_csv(output_data_dir/"iso_canopy_height.csv")